Python for Data Analysis, Vilnius University, 2024

# HW3: Training a successful ML model

There are 30 points to collect, and 5 bonus points.

In this homework, you need to write all code from scratch. There are minimal pre-written `assert` statements. Main rules for a successful submission:

**1. You can only import libraries from this list (but you won't need all of them):**

- Native Python libraries (e.g. itertools, re)
- Numpy
- Pandas
- Matplotlib
- Seaborn
- Scipy
- Scikit-Learn
- tqdm

**2. Do not leave unused code and ensure the overall structure of the Notebook is clean and readable, as you are given much more freedom this time. Do not repeat same code or create same variables more than 2 times.**

**3. Explain your choices (e.g. in comments).** Some code may be self-explanatory, like renaming columns or printing out some statistics. But there are things which should always be explained, such as:
- Any hardcoded values such as chosen thresholds or lists of column names;
- Removal of columns or samples;
- Imported functions which were not mentioned during lectures;
- Any additional steps which were not requested in tasks.

Don't hesitate to contact me or Martynas if you are stuck.

## The goal

Some researchers who collected data for the Snail Lab complained that measuring a sample takes a lot of time and is quite difficult. Sometimes snails are found on single leaves, not on whole plants, making several measurements impossible to make. Moreover, one should be careful when measuring snail features, because if they hide inside their shell, the researcher is forced to wait until the snail fully appears again.

An intern from the Snail Lab suggested that they could use AI to solve this issue. After several discussions, they decided to train a model which would predict the genus of a snail from only easy-to-measure features. The intern prepared some useful data but then got sick. So, the Snail Lab asks for your help once more.

You need to train a Logistic regression-based model to predict snail genus. For that, use data available here:

https://raw.githubusercontent.com/Tallivm/vu-python/refs/heads/main/hw3_data.csv

- Do not use any other datasets (e.g. from previous homeworks).

In [ ]:
import pandas as pd
# put all your imports into this cell

This variable contains the name of the column you need to predict. Use it through this Notebook instead of writing the column name itself. Do not rewrite it.

In [ ]:
to_predict = 'snail_genus'

## Task 1: EDA (exploratory data analysis)
*(3 points in total)*

- Write code to load `'hw3_data.csv'` into the variable `raw` as a DataFrame. Make sure to not use global paths (e.g. `"C://Users/JonasJonas/Desktop/vel_sraiges/hw3_data.csv"`) or paths which obviously won't work on another computers (e.g. `"folderiux/hw3_data.csv"`).

The Snail Lab assured you that this time the data should be completely clean. However, you always need to check it for yourself.
- Check if there are any NaNs.
- Check if mininum, maximum, and average values of numerical columns look OK. For example, if the measurement is about length, it cannot contain negative or zero values.
- Print out unique values and their counts in categorical columns.
- Plot a correlation matrix for numerical values. Make sure to follow the standard steps to make the plot easy to read (as mentioned in HW2, e.g. use a diverging colormap).

In [ ]:
# your code

In [ ]:
assert type(raw) == pd.DataFrame
assert to_predict in raw

## Task 2. Data transformation and scaling

There are some things to do before model training. Make sure that you make all changes in the new DataFrame named `transformed`.
- *(1 point)* Convert the values of snail genera into integers.
- *(2 points)* Logistic regression is sensitive to data variance. You need to standardize continuous features. Use a relevant scaler from Scikit-learn.
- *(1 point)* Obtain the same result by writing your own scaler function without Scikit-learn, using only math operators.
- *(2 points)* Categorical columns should not be standardized. Instead, they should be one-hot encoded. Use relevant function from either Scikit-learn or Pandas.
  - Make sure that from each categorical column with N unique values you get N-1 columns with integer values.
  - Do not encode snail genera like this.
- *(2 points)* Obtain the same result by writing your own one-hot encoding function.

Make sure `transformed` has correct column names. Print out 5 random samples from it.

In [ ]:
# your code

In [ ]:
assert raw.shape[1] < transformed.shape[1]
assert transformed[to_predict].dtype == int
assert 2 in transformed[to_predict]

## Task 3.

- *(1 point)* Split `transformed` into `X` (features to train the model with) and `y` (feature to be predicted).
  - `X` should have 2 dimensions and `y` only one dimension.

- *(3 points)* Train a Logistic regression model from Scikit-learn on transformed data to predict `y`.
  - For reproducibility, fix the random seed of the model.
  - Print out its **accuracy**.

- *(2 points)* Obtain model predictions of snail genera and plot a confusion matrix.

- *(1 point)* Obtain and print out the **precision** metric for each snail genus. Which snail genus is the easiest to predict?

In [ ]:
# your code

In [ ]:
assert X.ndim == 2
assert y.ndim == 1

## Task 4.

Someone from the lab theorized that knowing the genus of the plant which the leaf belongs to should help predict the genus of the snail, as certain snails are attracted to certain plants. However, the dataset does not contain plant names, and either way, plant identification requires additional time and skill.

Instead, you can use unsupervised learning to cluster plant features and use this information as a new feature.

- *(1 point)* Create a separate DataFrame `plants` containing only plant features. **This is the only place in the whole Notebook where it is allowed to write column names as strings instead of obtaining them automatically.**
  - Use transformed features for this step.

- *(5 points)* Collect new features: generate a dictionary `plant_clusters` with **at least** 20 elements which are Numpy arrays containing differently predicted plant clusters. Use Affinity Propagation, DBSCAN, and **at least** one more clustering method of your choice from [this Scikit-learn page](https://scikit-learn.org/1.5/modules/clustering.html).
  - Different clustering methods use different parameters. Run each clustering method several times with different parameters (no need to change every possible parameter, only the main ones). For example, DBSCAN has `eps` and `min_samples` parameters worth trying.
  - Use `for` loops; `itertools` can also be useful here.
  - Ensure the keys of `plant_clusters` dictionary are informative, i.e. they should tell which clustering method and what parameters were used. Create those keys automatically using string formatting.
  - The number of clusters in single clustering can be anywhere from 2 to 50.

In [ ]:
# your code

In [ ]:
assert type(plant_clusters) == dict
assert len(plant_clusters) >= 20
assert len(set([len(x) for x in plant_clusters.values()])) == 1

## Task 5.

For each plant clustering result from Task 4, check if it improves the accuracy of logistic regression.

- *(3 points)* For each feature in `plant_clusters`:
  - Create a new variable `XX` containing the `X` from Task 3 but joined with this new feature. If the shape of `X` was (M, N), then the shape of `XX` should be (M, N+1).
  - Create and fit a new logistic regression model on `XX` and the same `y` from Task 3.
  - Obtain model accuracy.
  - Print out the name of the added feature (key from `plant_clusters`) and the accuracy of the resulting predictions.

- *(2 points)* Choose the model with highest accuracy and plot a confusion matrix for it.
  - The best model should be selected automatically by saving all accuracy values and finding the highest one!

- *(1 point)* Print out precision metric again for each snail genus. Which genus is the easiest to predict with the new model? Calculate which genus got the biggest improvement in prediction.

In [ ]:
# your code

## Bonus: Task 6.

*(5 points)* By using any models from Scikit-learn, get a model which can predict snail genera with accuracy higher than 90%. For this task, you are free to try anything you want, including new feature generation or creating pipelines of models. The only rules are:
- You cannot use any other dataset files except for `hw3_data.csv`.
- The result should be reproducible (use fixed seeds for all random states).
- The column with snail genera should not be used in any way for model training. So, for example, if you use PCA on dataset, make sure to drop this column beforehand so its information won't be leaked into the result.

In [ ]:
# bonus code